In [3]:
import numpy as np
# import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Flatten, Dropout
from collections import Counter
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import pickle as pck
from nltk.tokenize import word_tokenize

In [1]:
text = ''
with open('hp.txt', 'r') as f:
    for line in f:
        if line != '':
            text += line + ' '

char_to_replace = ['.',',','!','?',':',';','(',')','-','\n','\\','\'']
for char in char_to_replace:
    text = text.translate({ord(char):None})

In [4]:
words = word_tokenize(text.lower())
# words = [word.lower() for word in text.split()]
counter = Counter(words)
total_words = len(words)
sorted_words = counter.most_common(total_words)
vocab = {w:i+1 for i,(w,c) in enumerate(sorted_words)}
inv_vocab = {i+1:w for i,(w,c) in enumerate(sorted_words)}

In [55]:
text_num = []
for word in words:
    text_num.append(vocab[word])

In [56]:
X = []
y = []
seq_len = 100
for i in range(len(words) - seq_len):
    seq_in = text_num[i:i+seq_len]
    seq_out = text_num[i+seq_len]
    X.append(seq_in)
    y.append(seq_out)

n_patterns = len(X)
print("number of sequences: ", n_patterns)

number of sequences:  82274


In [57]:
Xr = np.reshape(X, (n_patterns, seq_len, 1))
Xr = Xr / float(len(words))
yr = np_utils.to_categorical(y)

In [58]:
model = Sequential()
model.add(LSTM(256, input_shape=(Xr.shape[1], Xr.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(yr.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy')
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [63]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

print(Xr.shape)
print(yr.shape)
model.fit(Xr, yr, batch_size=256, epochs=100, validation_split=0.4)

(82274, 100, 1)
(82274, 6042)
Epoch 1/100
193/193 [==============================] - 392s 2s/step - loss: 6.4964 - val_loss: 6.4982
Epoch 2/100
193/193 [==============================] - 368s 2s/step - loss: 6.4843 - val_loss: 6.5425
Epoch 3/100
193/193 [==============================] - 383s 2s/step - loss: 6.4795 - val_loss: 6.5508
Epoch 4/100
193/193 [==============================] - 390s 2s/step - loss: 6.4770 - val_loss: 6.5636
Epoch 5/100
193/193 [==============================] - 389s 2s/step - loss: 6.4733 - val_loss: 6.5608
Epoch 6/100
193/193 [==============================] - 380s 2s/step - loss: 6.4727 - val_loss: 6.5682
Epoch 7/100
193/193 [==============================] - 368s 2s/step - loss: 6.4703 - val_loss: 6.5665
Epoch 8/100
193/193 [==============================] - 360s 2s/step - loss: 6.4682 - val_loss: 6.5744
Epoch 9/100
193/193 [==============================] - 356s 2s/step - loss: 6.4677 - val_loss: 6.5727
Epoch 10/100
193/193 [==============================

KeyboardInterrupt: 

In [ ]:
# with open("model.pck", 'wb') as f:
#     pck.dump(model, f)

In [66]:
seq = text_num[:seq_len]
text = ' '.join(words[:seq_len])
for i in range(100):
    x = np.reshape(seq, (1, seq_len, 1))
    x = x / float(len(words))
    p = model.predict(x)
    index = np.argmax(p)
#     print(index)
    seq.append(index)
    seq = seq[1:]
    text += ' ' + inv_vocab[index]

In [67]:
text

"harry potter and the sorcerers stone chapter one the boy who lived mr and mrs dursley of number four privet drive were proud to say that they were perfectly normal thank you very much they were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense mr dursley was the director of a firm called grunnings which made drills he was a big beefy man with hardly any neck although he did have a very large mustache mrs dursley was thin and blonde and had nearly twice the usual and the the the the the the the the the the the the the the the the the the the the the the the the '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''"